In [1]:
import textract
import pandas as pd
import os
import json

In [2]:
resume_folder_path = '/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/Eximius Employees Resumes PDF Format'
skillsheet_file_path = '/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/Key Word Search.xlsx'

In [4]:
skill_sheet = pd.read_excel(skillsheet_file_path, header=[1, 2], dtype=str)
skill_sheet = skill_sheet.drop(skill_sheet.columns[0], axis=1)

In [5]:
skills_dict = {}
all_skills = []
for label, label_data in skill_sheet.iteritems():
    skills_dict[label] = label_data.dropna()
    all_skills.extend(label_data.dropna().tolist())

In [14]:
all_resume_data = []
cnt = 0
for each_file in os.listdir(resume_folder_path):
    each_resume_dict = {}
    each_file_path = os.path.join(resume_folder_path, each_file)
    each_file_str = textract.process(each_file_path, encoding='ascii').decode()
    each_resume_dict['Filename'] = each_file
    each_resume_dict['ResumeString'] = each_file_str
    skills_found = [each_skill.strip() for each_skill in all_skills if each_skill.strip() in each_file_str]
    each_resume_dict['SkillsFound'] = list(set(skills_found))
    
    yearsOfExp = each_file_str.split()[0].split('/')[-1]
    each_resume_dict['YearsOfExperience'] = yearsOfExp
    
    all_resume_data.append(each_resume_dict)
    cnt += 1

In [7]:
print('TotalFileCount', cnt)

TotalFileCount 208


In [15]:
with open('/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/resume_data.json', 'w') as f:
    json.dump(all_resume_data, f, indent=4)

In [9]:
def get_sorted_resumes(user_query):
    
    resume_skill_count = {}
    
    all_data = json.load(open('/home/ayusheea/Documents/ML_VirtualEnvs/ResumeSelectionProject/SkillMapping/data/resume_data.json', 'r'))
    
    for each_resume_data in all_data:
        skills_matched_cnt = 0
        skills_matched = []
        resume_skills = each_resume_data['SkillsFound']
        for each_query in user_query.split(','):
            each_query = each_query.strip()
    #         print(each_query, resume_skills)  
            if each_query in resume_skills:
    #             print('inside if')
    #             print('inside', each_query, resume_skills)
                skills_matched_cnt += 1
                skills_matched.append(each_query)
    #     print(skills_matched_cnt)
        if skills_matched_cnt>0:
            resume_skill_count[each_resume_data['Filename']] = (skills_matched_cnt, skills_matched)  
    sorted_resumes = sorted(resume_skill_count.keys(), key=lambda x: resume_skill_count[x][0], reverse=True)
    return sorted_resumes, resume_skill_count

In [10]:
user_query = 'Embedded C, Firmware, BSP, SATA, CAN, Python, Autosar, IPC, Framework, Android'

In [17]:
get_sorted_resumes(user_query)[1]

{'Eximius Design - Swetabh Suman.pdf': (4,
  ['Python', 'IPC', 'Framework', 'Android']),
 'Eximius Design - Priyadarshini Satapathy.pdf': (3,
  ['Firmware', 'SATA', 'Python']),
 'Eximius Design - Bhanu Kumar.pdf': (3, ['Firmware', 'SATA', 'IPC']),
 'Eximius Design - Dileep Prem Kumar Katta.pdf': (3,
  ['Firmware', 'SATA', 'Framework']),
 'Eximius Design - Smisha C B.pdf': (2, ['Framework', 'Android']),
 'Eximius Design - Rahul Kumar Choube.pdf': (6,
  ['Embedded C', 'BSP', 'Python', 'IPC', 'Framework', 'Android']),
 'Eximius Design - Pradeep Kumar K.pdf': (2, ['Firmware', 'Python']),
 'Eximius Design - Sudhindra Gubbi Raj.pdf': (2, ['Python', 'Android']),
 'Eximius Design - Karthikeyan Velayudham.pdf': (3,
  ['Firmware', 'SATA', 'Python']),
 'Eximius Design - Shashwat Pandey.pdf': (3, ['BSP', 'Python', 'Framework']),
 'Eximius Design - Anamika Singh.pdf': (2, ['Python', 'Android']),
 'Eximius Design - Y Muhammadi Khanam.pdf': (1, ['BSP']),
 'Eximius Design - Arumugam Palaniyandi.pdf': 